In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [9]:
explainer_map = {'conceptx': 'ConceptX', 'aconceptx': 'AntonymConceptX', 'conceptx_r': 'ConceptX-R', 'conceptx_a': 'ConceptX-A', 'conceptshap': 'ConceptSHAP', 'tokenshap': 'TokenSHAP', 'random': 'Random'}
explainer_order = ["Random", "TokenSHAP", "ConceptSHAP", "ConceptX", 'AntonymConceptX', "ConceptX-R", "ConceptX-A"]
MODEL_NAMES = {"gpt2": "GPT-2", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o-mini", "llama-3-3b": "Llama-3.2-3B"}

In [10]:
save_dir = "/cluster/home/kamara/conceptx"
seed_ = 0

In [20]:
folder_path = os.path.join(save_dir, f"results/classification")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print("parts: ", parts)
        
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        df["explainer"] = parts[dataset_idx + 2]
        
        if len(parts) > dataset_idx + 4:
            df["baseline"] = parts[dataset_idx + 3]
            seed_idx = dataset_idx + 4
        else:
            df["baseline"] = None
            seed_idx = dataset_idx + 3

        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)
# Update 'explainer' column based on 'baseline' conditions
df_all["explainer"] = df_all["explainer"] + df_all["baseline"].map({"reference": "_r", "aspect": "_a"}).fillna("")
df_all

parts:  ['classification', 'batch', '8', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '9', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '1', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '3', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '4', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '2', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '0', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '6', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '5', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '7', 'sst2', 'llama-3-3b', 'random', '0.csv']
parts:  ['classification', 'batch', '8', 'sst2', 'llama-3-3b', 'tokenshap', '0.csv']
parts:  ['classification', 'batch', '9', 'sst2', 'llama-3-3b', 'tokenshap', '0.csv']
parts:  ['

,id,input,p0,p_highest,p_label,aspect,highest_token,label,batch,dataset,model,explainer,baseline,seed
0,2958,assumes you are n't very bright,0.644754,0.761072,0.413600,negative,assumes,bright,8,sst2,llama-3-3b,random,None,0
1,2966,i hated myself in the morning .,0.933341,0.911667,0.047257,negative,in,hated,8,sst2,llama-3-3b,random,None,0
2,2967,with efficiency and an affection for the period,0.346902,0.173281,0.173281,positive,affection,affection,8,sst2,llama-3-3b,random,None,0
3,2968,confection that 's pure entertainment,0.849513,0.880429,0.817693,positive,that,entertainment,8,sst2,llama-3-3b,random,None,0
4,2969,bearing the unmistakable stamp of authority,0.127977,0.069199,0.148326,positive,stamp,authority,8,sst2,llama-3-3b,random,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34249,3526,Excellent he will never know lmao,0.473370,0.070706,0.070706,positive,Excellent,Excellent,5,sentiment,gpt4o-mini,aconceptx,None,0
34250,3528,am little bit upset ppl is jerk,0.896111,0.804609,0.788253,negative,jerk,upset,5,sentiment,gpt4o-mini,aconceptx,None,0
34251,3530,This class is really long and Im really gettin...,0.858720,0.870074,0.891205,negative,getting,hungry,5,sentiment,gpt4o-mini,aconceptx,None,0
34252,3536,Happy Mothers Day A rose for YOU,0.984503,0.921839,0.921839,positive,Happy,Happy,5,sentiment,gpt4o-mini,aconceptx,None,0


In [21]:
### Size of dataset for model
model_list = ["llama-3-3b", "gemma-3-4b", "gpt4o-mini"]
df = df_all[df_all["model"].isin(model_list)]
df_stats = df.groupby(["dataset", "model", "explainer"])["id"].nunique().reset_index()

In [22]:
dataset = "sentiment"
df_stats[df_stats['dataset']==dataset]

,dataset,model,explainer,id
0,sentiment,gemma-3-4b,aconceptx,927
1,sentiment,gemma-3-4b,conceptshap,948
2,sentiment,gemma-3-4b,conceptx,938
3,sentiment,gemma-3-4b,conceptx_a,938
4,sentiment,gemma-3-4b,random,996
5,sentiment,gemma-3-4b,tokenshap,996
6,sentiment,gpt4o-mini,aconceptx,919
7,sentiment,gpt4o-mini,conceptshap,937
8,sentiment,gpt4o-mini,conceptx,929
9,sentiment,gpt4o-mini,conceptx_a,928


In [23]:
# Convert relevant columns to numeric
df_all["p0"] = pd.to_numeric(df_all["p0"], errors="coerce")
df_all["p_highest"] = pd.to_numeric(df_all["p_highest"], errors="coerce")
df_all["p_label"] = pd.to_numeric(df_all["p_label"], errors="coerce")


In [24]:
# Filter only by dataset and p0 threshold (no longer filtering by model)
df = df_all[(df_all["dataset"] == dataset)]
df = df[df["p0"] > 0.5]

# Compute diffs
df["p0_p_highest_diff"] = df["p0"] - df["p_highest"]
df["p0_p_label_diff"] = df["p0"] - df["p_label"]

# Compute stats: group by explainer and model
stats = df.groupby(["explainer", "model"])[["p0_p_highest_diff"]].agg(["mean", "var"])

# Compute stats: group by explainer and model
df_label = df[['model', 'explainer', "p0_p_label_diff"]]
df_label = df_label.rename(columns={'p0_p_label_diff': 'p0_p_highest_diff'})

stats_label = df_label.groupby(["model"])[["p0_p_highest_diff"]].agg(["mean", "var"])
# Step 2: Add new level to index to create the new explainer row
stats_label["explainer"] = "gpt4o-mini_explainer"
stats_label = stats_label.set_index("explainer", append=True)
stats_label.index = stats_label.index.reorder_levels(["explainer", "model"])

# Step 3: Concatenate with df1
df_combined = pd.concat([stats, stats_label])

# Step 4: Optional: sort for clarity
df_combined = df_combined.sort_index()


# Flatten multi-index columns
df_combined.columns = [f"{model}_{stat}" for metric, stat in df_combined.columns for model in [metric.split("_diff")[0]]]

# Pivot so each model becomes a column group
df_combined = df_combined.unstack("model")

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "aconceptx", "conceptx_a", "gpt4o-mini_explainer"]
explainer_map = {
    'conceptx': 'ConceptX',
    'aconceptx': 'AntonymConceptX',
    'conceptx_r': 'ConceptX-R',
    'conceptx_a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random',
    'gpt4o-mini_explainer': 'GPT-4o mini'
}
df_combined = df_combined.rename(index=explainer_map)
df_combined = df_combined.reindex([explainer_map.get(e, e) for e in desired_order])

df_combined.columns = df_combined.columns.swaplevel()
df_combined = df_combined.sort_index(axis=1)
df_combined

model                  gemma-3-4b                         gpt4o-mini  \
                p0_p_highest_mean p0_p_highest_var p0_p_highest_mean   
explainer                                                              
Random                   0.074230         0.031221          0.085116   
TokenSHAP                0.273535         0.096169          0.305063   
ConceptSHAP              0.214924         0.082951          0.247707   
ConceptX                 0.189259         0.077677          0.196752   
AntonymConceptX          0.249684         0.093696          0.218964   
ConceptX-A               0.151431         0.069351          0.206929   
GPT-4o mini              0.339263         0.093256          0.337172   

model                                   llama-3-3b                   
                p0_p_highest_var p0_p_highest_mean p0_p_highest_var  
explainer                                                            
Random                  0.037680          0.078241         0.032885  
TokenSHAP               0.096900          0.099612         0.041839  
ConceptSHAP             0.092867          0.111069         0.055956  
ConceptX                0.080480          0.119957         0.053078  
AntonymConceptX         0.085934          0.142696         0.061943  
ConceptX-A              0.082125          0.126016         0.054513  
GPT-4o mini             0.092645          0.342024         0.093421

In [19]:

MODEL_NAMES = {"gpt2": "GPT-2", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o mini", "llama-3-3b": "Llama-3.2-3B"}
model_map = MODEL_NAMES
model_order = ["Gemma-3-4B", "Llama-3.2-3B", "GPT-4o mini"]


# Rename models in column MultiIndex
df_combined.columns = pd.MultiIndex.from_tuples([
    (model_map.get(model, model), metric) for model, metric in df_combined.columns
])

# Reorder columns according to model_order
df_combined = df_combined[model_order]
df_combined

Gemma-3-4B                       Llama-3.2-3B  \
                p0_p_highest_mean p0_p_highest_var p0_p_highest_mean   
explainer                                                              
Random                   0.134776         0.047699          0.134776   
TokenSHAP                0.343397         0.091753          0.128383   
ConceptSHAP              0.281959         0.084161          0.179892   
ConceptX                 0.238014         0.080016          0.172192   
AntonymConceptX          0.298050         0.087347          0.174077   
ConceptX-A               0.193401         0.069601          0.175203   
GPT-4o mini              0.404809         0.079446          0.404598   

                                       GPT-4o mini                   
                p0_p_highest_var p0_p_highest_mean p0_p_highest_var  
explainer                                                            
Random                  0.047699          0.133208         0.047643  
TokenSHAP               0.047355          0.347770         0.086186  
ConceptSHAP             0.063703          0.291164         0.082323  
ConceptX                0.058694          0.258762         0.083268  
AntonymConceptX         0.059246          0.245704         0.077980  
ConceptX-A              0.061259          0.272881         0.076672  
GPT-4o mini             0.079555          0.404363         0.079618

In [12]:
# Filter and compute diffs
df = df_all[(df_all["dataset"] == dataset) & (df_all["model"] == model_name)]
df = df[df["p0"] > 0.5]

df["p0_p_highest_diff"] = df["p0"] - df["p_highest"]
df["p0_p_label_diff"] = df["p0"] - df["p_label"]

# Group stats by explainer
stats = df.groupby("explainer")[["p0_p_highest_diff"]].agg(["mean", "var"])

# Compute overall mean/var for p0_p_label_diff
label_diff_mean = df["p0_p_label_diff"].mean()
label_diff_var = df["p0_p_label_diff"].var()

# Create a row with NaNs for p0_p_highest_diff, and actual values for p0_p_label_diff
import numpy as np
new_row = pd.DataFrame({
    ("p0_p_highest_diff", "mean"): [label_diff_mean],
    ("p0_p_highest_diff", "var"): [label_diff_var]
}, index=["gpt4o-mini"])

# Append to the existing stats
stats = pd.concat([stats, new_row])
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "conceptx_a", "gpt4o-mini"]
stats = stats.reindex(desired_order)
explainer_map = {'conceptx': 'ConceptX', 'conceptx_r': 'ConceptX-R', 'conceptx_a': 'ConceptX-A', 'conceptshap': 'ConceptSHAP', 'tokenshap': 'TokenSHAP', 'random': 'Random'}
# Apply renaming to the index
stats = stats.rename(index=explainer_map)
# Print or save
print(stats)

            p0_p_highest_diff          
                         mean       var
Random               0.134776  0.047699
TokenSHAP            0.128383  0.047355
ConceptSHAP          0.179892  0.063703
ConceptX             0.172192  0.058694
ConceptX-A           0.175203  0.061259
gpt4o-mini           0.404532  0.079533
